In [1]:
import json
import numpy as np
import hnswlib
import networkx as nx
from collections import defaultdict

In [2]:
import scipy.sparse as sp

"""
Disclaimer: functions defined from lines 15 to 36 in this file come from 
tkipf/gae original repository on Graph Autoencoders. Moreover, the
mask_test_edges function is borrowed from philipjackson's mask_test_edges 
pull request on this same repository.
"""

def sparse_to_tuple(sparse_mx):
    if not sp.isspmatrix_coo(sparse_mx):
        sparse_mx = sparse_mx.tocoo()
    coords = np.vstack((sparse_mx.row, sparse_mx.col)).transpose()
    values = sparse_mx.data
    shape = sparse_mx.shape
    return coords, values, shape

def preprocess_graph(adj):
    adj = sp.coo_matrix(adj)
    adj_ = adj + sp.eye(adj.shape[0])
    degree_mat_inv_sqrt = sp.diags(np.power(np.array(adj_.sum(1)), -0.5).flatten())
    adj_normalized = adj_.dot(degree_mat_inv_sqrt).transpose().dot(degree_mat_inv_sqrt)
    return sparse_to_tuple(adj_normalized)

def construct_feed_dict(adj_normalized, adj, features, placeholders):
    # Construct feed dictionary
    feed_dict = dict()
    feed_dict.update({placeholders['features']: features})
    feed_dict.update({placeholders['adj']: adj_normalized})
    feed_dict.update({placeholders['adj_orig']: adj})
    return feed_dict

def mask_test_edges(adj, test_percent=1., val_percent=0.):
    """ Randomly removes some edges from original graph to create
    test and validation sets for link prediction task
    :param adj: complete sparse adjacency matrix of the graph
    :param test_percent: percentage of edges in test set
    :param val_percent: percentage of edges in validation set
    :return: train incomplete adjacency matrix, validation and test sets
    """
    # Remove diagonal elements
    adj = adj - sp.dia_matrix((adj.diagonal()[None, :], [0]), shape=adj.shape)
    adj.eliminate_zeros()
    # Check that diag is zero:
    #assert adj.diagonal().sum() == 0

    edges_positive, _, _ = sparse_to_tuple(adj)
    # Filtering out edges from lower triangle of adjacency matrix
    edges_positive = edges_positive[edges_positive[:,1] > edges_positive[:,0],:]
    # val_edges, val_edges_false, test_edges, test_edges_false = None, None, None, None

    # number of positive (and negative) edges in test and val sets:
    num_test = int(np.floor(edges_positive.shape[0] / (100. / test_percent)))
    num_val = 0

    # sample positive edges for test and val sets:
    edges_positive_idx = np.arange(edges_positive.shape[0])
    np.random.shuffle(edges_positive_idx)
    val_edge_idx = edges_positive_idx[:num_val]
    test_edge_idx = edges_positive_idx[num_val:(num_val + num_test)]
    test_edges = edges_positive[test_edge_idx] # positive test edges
    val_edges = edges_positive[val_edge_idx] # positive val edges
    train_edges = np.delete(edges_positive, np.hstack([test_edge_idx, val_edge_idx]), axis = 0) # positive train edges

    # the above strategy for sampling without replacement will not work for
    # sampling negative edges on large graphs, because the pool of negative
    # edges is much much larger due to sparsity, therefore we'll use
    # the following strategy:
    # 1. sample random linear indices from adjacency matrix WITH REPLACEMENT
    # (without replacement is super slow). sample more than we need so we'll
    # probably have enough after all the filtering steps.
    # 2. remove any edges that have already been added to the other edge lists
    # 3. convert to (i,j) coordinates
    # 4. swap i and j where i > j, to ensure they're upper triangle elements
    # 5. remove any duplicate elements if there are any
    # 6. remove any diagonal elements
    # 7. if we don't have enough edges, repeat this process until we get enough
    positive_idx, _, _ = sparse_to_tuple(adj) # [i,j] coord pairs for all true edges
    positive_idx = positive_idx[:,0]*adj.shape[0] + positive_idx[:,1] # linear indices
    test_edges_false = np.empty((0,2),dtype='int64')
    idx_test_edges_false = np.empty((0,),dtype='int64')

    while len(test_edges_false) < len(test_edges):
        # step 1:
        idx = np.random.choice(adj.shape[0]**2, 2*(num_test - len(test_edges_false)), replace = True)
        # step 2:
        idx = idx[~np.in1d(idx, positive_idx, assume_unique = True)]
        idx = idx[~np.in1d(idx, idx_test_edges_false, assume_unique = True)]
        # step 3:
        rowidx = idx // adj.shape[0]
        colidx = idx % adj.shape[0]
        coords = np.vstack((rowidx,colidx)).transpose()
        # step 4:
        lowertrimask = coords[:,0] > coords[:,1]
        coords[lowertrimask] = coords[lowertrimask][:,::-1]
        # step 5:
        coords = np.unique(coords, axis = 0) # note: coords are now sorted lexicographically
        np.random.shuffle(coords) # not anymore
        # step 6:
        coords = coords[coords[:,0] != coords[:,1]]
        # step 7:
        coords = coords[:min(num_test, len(idx))]
        test_edges_false = np.append(test_edges_false, coords, axis = 0)
        idx = idx[:min(num_test, len(idx))]
        idx_test_edges_false = np.append(idx_test_edges_false, idx)

    val_edges_false = np.empty((0,2), dtype = 'int64')
    idx_val_edges_false = np.empty((0,), dtype = 'int64')
#     while len(val_edges_false) < len(val_edges):
#         # step 1:
#         idx = np.random.choice(adj.shape[0]**2, 2*(num_val - len(val_edges_false)), replace = True)
#         # step 2:
#         idx = idx[~np.in1d(idx, positive_idx, assume_unique = True)]
#         idx = idx[~np.in1d(idx, idx_test_edges_false, assume_unique = True)]
#         idx = idx[~np.in1d(idx, idx_val_edges_false, assume_unique = True)]
#         # step 3:
#         rowidx = idx // adj.shape[0]
#         colidx = idx % adj.shape[0]
#         coords = np.vstack((rowidx,colidx)).transpose()
#         # step 4:
#         lowertrimask = coords[:,0] > coords[:,1]
#         coords[lowertrimask] = coords[lowertrimask][:,::-1]
#         # step 5:
#         coords = np.unique(coords, axis = 0) # note: coords are now sorted lexicographically
#         np.random.shuffle(coords) # not any more
#         # step 6:
#         coords = coords[coords[:,0] != coords[:,1]]
#         # step 7:
#         coords = coords[:min(num_val, len(idx))]
#         val_edges_false = np.append(val_edges_false, coords, axis = 0)
#         idx = idx[:min(num_val, len(idx))]
#         idx_val_edges_false = np.append(idx_val_edges_false, idx)

    # sanity checks:
#     train_edges_linear = train_edges[:,0]*adj.shape[0] + train_edges[:,1]
#     test_edges_linear = test_edges[:,0]*adj.shape[0] + test_edges[:,1]
#     assert not np.any(np.in1d(idx_test_edges_false, positive_idx))
#     assert not np.any(np.in1d(idx_val_edges_false, positive_idx))
#     assert not np.any(np.in1d(val_edges[:,0]*adj.shape[0]+val_edges[:,1], train_edges_linear))
#     assert not np.any(np.in1d(test_edges_linear, train_edges_linear))
#     assert not np.any(np.in1d(val_edges[:,0]*adj.shape[0]+val_edges[:,1], test_edges_linear))

    # Re-build adj matrix
    data = np.ones(train_edges.shape[0])
    adj_train = sp.csr_matrix((data, (train_edges[:, 0], train_edges[:, 1])), shape=adj.shape)
    adj_train = adj_train + adj_train.T
    return adj_train, val_edges, val_edges_false, test_edges, test_edges_false

## Reading Embeddings and creating Indexer for NN search

In [ ]:
IDList = []                                # List of paper IDs
NNList = []                                # List of list, NNList[i]: NNs to paper whose id is IDList[i]
embeddings = []                            # Embeddings read from the input file

with open('./data/dblp_Abstract_2Thresholded_USE_Trans_Embeddings.json', 'r') as file:
    for line in file:
        data = json.loads(line)
        paperID = data['id']
        embedding = data['embedding']
        IDList.append(paperID)
        embeddings.append(embedding)

In [ ]:
numElements = len(IDList)
dimension = len(embeddings[0])
embeddings = np.asarray(embeddings)
data_labels = np.arange(numElements)

In [ ]:
p = hnswlib.Index(space = 'cosine', dim = dimension) # possible options are l2, cosine or ip

# Initing index - the maximum number of elements should be known beforehand
p.init_index(max_elements = numElements, ef_construction = 200, M = 20)

# Element insertion (can be called several times):
p.add_items(embeddings, data_labels)

# Controlling the recall by setting ef:
p.set_ef(50) # ef should always be > k

# Query dataset, k - number of closest elements (returns 2 numpy arrays)
labels, _ = p.knn_query(embeddings, k = 5)

In [ ]:
index_path='./models/USETranshnswlib.bin'
print("Saving index to '%s'" % index_path)
p.save_index("./models/USETranshnswlib.bin")
del p

In [ ]:
p = hnswlib.Index(space='cosine', dim=dimension)  # the space can be changed - keeps the data, alters the distance function.

# Increase the total capacity (max_elements), so that it will handle the new data
p.load_index("./models/USETranshnswlib.bin", max_elements = numElements)
labels, _ = p.knn_query(embeddings, k = 4)
del p
del embeddings
del data_labels

## Examples of NN obtained using the Text Embeddings

In [ ]:
titles = []
with open('./data/dblp_AIpapers2Thresholded.json', 'r') as file:
    for line in file:
        data = json.loads(line)
        titles.append(data['title'])

In [ ]:
count = 5
for i in range(count):
    print('Paper: ', titles[i])
    print('Nearest Papers: ', [titles[ind] for ind in labels[i] if ind != i])
    print('\n')

## Building Adjacency List for Node Embeddings

### Creating Citation Adjacency List

In [3]:
adjList = defaultdict(set)                          # Convert set to list later for node2vec, set: to handle duplicates
with open('./data/dblp_AIpapers2Thresholded.json', 'r') as file:
    for line in file:
        data = json.loads(line)
        paperID = data['id']
        references = data.get('references', [])
        for referencedPaper in references:
            adjList[paperID].add(referencedPaper)
            adjList[referencedPaper].add(paperID)

### Augmenting Adj List with FastText NNs

In [ ]:
nnToKeep = 4
id = 0
for label in labels:
    paperID = IDList[id]
    label = [IDList[index] for index in label if index != id]
    if (len(label) > nnToKeep):
        del label[nnToKeep:]
    for referencedPaper in label:
        adjList[paperID].add(referencedPaper)
        adjList[referencedPaper].add(paperID)
    id += 1

### Creating NetworkX Graph and reporting graph statistics

In [4]:
adjList = {key: list(values) for key, values in adjList.items()}
G = nx.from_dict_of_lists(adjList)

nnodes = G.number_of_nodes()
avgDegree = sum(d for n, d in G.degree()) / float(nnodes)
print('Number of nodes: ', nnodes, '. Number of edges: ', G.number_of_edges(), '. Avg Degree: ', avgDegree)

Number of nodes:  471633 . Number of edges:  5464345 . Avg Degree:  23.172021465843144


In [5]:
adj_sparse = nx.to_scipy_sparse_matrix(G)

In [6]:
# Perform train-test split
adj_train, val_edges, val_edges_false, test_edges, test_edges_false = mask_test_edges(adj_sparse)
print('Constructing new graph')
G_train = nx.from_scipy_sparse_matrix(adj_train) # new graph object with only non-hidden edges

Constructing new graph


In [23]:
G_train.number_of_nodes() == G.number_of_nodes()

True

In [7]:
nodes = list(G.nodes())

In [14]:
len(nodes)

471633

In [19]:
for adjV in adjList[u]:
    if adjV == v:
        print('Found')
    
## paperID mapping: G.nodes() are paperIDs one to one mapped with G_train.nodes which are 0 to |V| - 1
## test edges and test edges negative are integer

Found


## Node2Vec Embeddings

In [ ]:
from node2vec import Node2Vec
walkLength = 6
node2vec = Node2Vec(G_train, walk_length = walkLength)#, workers = 12, temp_folder = './data/tmp_data')
          

Computing transition probabilities:   4%|▍         | 18408/471633 [27:11<12:10:31, 10.34it/s]

In [ ]:
model = node2vec.fit()  # returns a gensim wv model

In [ ]:
outFileName = './models/node2vec_USE_2Citation_Embeddings_WL_' + str(walkLength) + '_NN_' + str(nnToKeep) + '.kv'
model.wv.save_word2vec_format(outFileName)

## Edge Prediction

In [11]:
def getEdgeEmbedding(embedding1, embedding2, policy='Hadamard'):
    if (policy=='Hadamard'):
        return embedding1 * embedding2
    elif(policy=='Avg'):
        return (embedding1 + embedding2) / 2

In [32]:
def average(lis):
    return (sum(lis) / len(lis))

In [12]:
word2VecMode = True

In [26]:
embeddingDict = dict()

if (word2VecMode):
    embeddingFileName = './data/dblpAbstract_2Thresholded_FT_Embeddings.json'
    with open(embeddingFileName, 'r') as file:
        for line in file:
            data = json.loads(line)
            embeddingDict[data['id']] = data['embedding']

In [37]:
            
X = []
edges = [*test_edges, *test_edges_false]
for edge in edges :
    if (word2VecMode):
        try:
            paperID1 = nodes[edge[0]]
        except:
            print(edge[0])
            break
        try:
            paperID2 = nodes[edge[1]]
        except:
            print(edge[1])
            break
        embedding1 = np.asarray(embeddingDict[paperID1])
        embedding2 = np.asarray(embeddingDict[paperID2])
    else:
        if edge[0] in model.wv.vocab:
            embedding1 = np.asarray(model.wv[edge[0]])
        else:
            embedding1 = np.asarray([0] * 128)
        if edge[1] in model.wv.vocab:
            embedding2 = np.asarray(model.wv[edge[1]])
        else:
            embedding2 = np.asarray([0] * 128)
    edgeEmbedding =  getEdgeEmbedding(embedding1, embedding2)
    X.append(edgeEmbedding)

Y = np.concatenate(np.ones(len(test_edges), dtype = int), np.zeros(len(test_edges_false), dtype=int))
if (word2VecMode):
    del embeddingDict    
X = np.asarray(X)
    

NameError: name 'embeddingDict' is not defined

In [30]:
from sklearn.neural_network import MLPClassifier
from sklearn.svm import LinearSVC
from sklearn.gaussian_process.kernels import RBF
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier
from sklearn.multiclass import OneVsRestClassifier


names = [
 "Random Forest", "Neural Net", "Logistic Regression", "Linear SVC" ]

classifiers = [
    RandomForestClassifier(verbose=True, n_jobs = -1),
    MLPClassifier(verbose=True, early_stopping=True),
    AdaBoostClassifier(),
    OneVsRestClassifier(BaggingClassifier(LinearSVC(),n_jobs = -1))]


In [35]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import KFold

kfold = KFold(n_splits=5, shuffle=True)
for name, clf in zip(names, classifiers):
    precScores = []
    recallScores = []
    f1Scores = []
    count = 1
    for train_index, test_index in kfold.split(X, Y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]
        print('Fitting: ', count)
        clf.fit(X_train, y_train)
        print('count ', count)
        y_pred = clf.predict(X_test)
        prec, recall, fscore, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')
        precScores.append(prec)
        recallScores.append(recall)
        f1Scores.append(fscore)
        count += 1
    print('Name', name,'. Avg Precision: ', average(precScores), '. Avg Recall: ', average(recallScores), '. Avg F-1 Score: ', average(f1Scores) )


Fitting:  1


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 48 concurrent workers.
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   15.4s finished
[Parallel(n_jobs=48)]: Using backend ThreadingBackend with 48 concurrent workers.
[Parallel(n_jobs=48)]: Done 100 out of 100 | elapsed:    0.1s finished


count  1
Fitting:  2


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 48 concurrent workers.
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   15.0s finished
[Parallel(n_jobs=48)]: Using backend ThreadingBackend with 48 concurrent workers.
[Parallel(n_jobs=48)]: Done 100 out of 100 | elapsed:    0.1s finished


count  2
Fitting:  3


...

count  5
Name Random Forest . Avg Precision:  0.7716315809855593 . Avg Recall:  0.769714307624117 . Avg F-1 Score:  0.7693101532464279
Fitting:  1
Iteration 1, loss = 0.69222773
Validation score: 0.500629
Iteration 2, loss = 0.68977166
Validation score: 0.559190
Iteration 3, loss = 0.68747817
Validation score: 0.631477
Iteration 4, loss = 0.68475911
Validation score: 0.589157
Iteration 5, loss = 0.68124036
Validation score: 0.679744
Iteration 6, loss = 0.67625042
Validation score: 0.659156
Iteration 7, loss = 0.66971378
Validation score: 0.693355
Iteration 8, loss = 0.66221698
Validation score: 0.698502
Iteration 9, loss = 0.65353430
Validation score: 0.698502
Iteration 10, loss = 0.64473392
Validation score: 0.699188
Iteration 11, loss = 0.63625275
Validation score: 0.712456
Iteration 12, loss = 0.62788696
Validation score: 0.716116
Iteration 13, loss = 0.61963378
Validation score: 0.708567
Iteration 14, loss = 0.61152296
Validation score: 0.727325
Iteration 15, loss = 0.60445811
Vali

Iteration 19, loss = 0.57842479
Validation score: 0.738762
Iteration 20, loss = 0.57277527
Validation score: 0.745854
Iteration 21, loss = 0.56713321
Validation score: 0.751687
Iteration 22, loss = 0.56232225
Validation score: 0.752259
Iteration 23, loss = 0.55823932
Validation score: 0.756720
Iteration 24, loss = 0.55361263
Validation score: 0.748141
Iteration 25, loss = 0.55029404
Validation score: 0.757406
Iteration 26, loss = 0.54603307
Validation score: 0.761638
Iteration 27, loss = 0.54259737
Validation score: 0.764383
Iteration 28, loss = 0.53966785
Validation score: 0.761752
Iteration 29, loss = 0.53719750
Validation score: 0.758321
Iteration 30, loss = 0.53417311
Validation score: 0.767014
Iteration 31, loss = 0.53091961
Validation score: 0.767929
Iteration 32, loss = 0.52888568
Validation score: 0.768844
Iteration 33, loss = 0.52687219
Validation score: 0.769873
Iteration 34, loss = 0.52431832
Validation score: 0.766785
Iteration 35, loss = 0.52265185
Validation score: 0.7687

Validation score: 0.784856
Iteration 59, loss = 0.49396801
Validation score: 0.786801
Iteration 60, loss = 0.49381789
Validation score: 0.785886
Iteration 61, loss = 0.49308003
Validation score: 0.779938
Iteration 62, loss = 0.49212865
Validation score: 0.785543
Iteration 63, loss = 0.49282856
Validation score: 0.786572
Iteration 64, loss = 0.49087048
Validation score: 0.788402
Iteration 65, loss = 0.49065242
Validation score: 0.786686
Iteration 66, loss = 0.49022405
Validation score: 0.787258
Iteration 67, loss = 0.48950704
Validation score: 0.785428
Iteration 68, loss = 0.48872699
Validation score: 0.788059
Iteration 69, loss = 0.48874785
Validation score: 0.789317
Iteration 70, loss = 0.48761253
Validation score: 0.789889
Iteration 71, loss = 0.48765170
Validation score: 0.789088
Iteration 72, loss = 0.48771012
Validation score: 0.784056
Iteration 73, loss = 0.48610274
Validation score: 0.788631
Iteration 74, loss = 0.48620546
Validation score: 0.785314
Iteration 75, loss = 0.486302

Validation score: 0.788974
Iteration 61, loss = 0.48975575
Validation score: 0.791033
Iteration 62, loss = 0.48819170
Validation score: 0.792177
Iteration 63, loss = 0.48806723
Validation score: 0.786572
Iteration 64, loss = 0.48689949
Validation score: 0.790347
Iteration 65, loss = 0.48717800
Validation score: 0.791833
Iteration 66, loss = 0.48756891
Validation score: 0.780281
Iteration 67, loss = 0.48556979
Validation score: 0.790575
Iteration 68, loss = 0.48513036
Validation score: 0.793664
Iteration 69, loss = 0.48456289
Validation score: 0.794235
Iteration 70, loss = 0.48445201
Validation score: 0.791948
Iteration 71, loss = 0.48418221
Validation score: 0.794007
Iteration 72, loss = 0.48294217
Validation score: 0.791605
Iteration 73, loss = 0.48324460
Validation score: 0.791833
Iteration 74, loss = 0.48312639
Validation score: 0.795150
Iteration 75, loss = 0.48184037
Validation score: 0.796065
Iteration 76, loss = 0.48179300
Validation score: 0.794922
Iteration 77, loss = 0.481032

Validation score: 0.781997
Iteration 112, loss = 0.47162728
Validation score: 0.781082
Iteration 113, loss = 0.47146377
Validation score: 0.786343
Iteration 114, loss = 0.47038736
Validation score: 0.785771
Iteration 115, loss = 0.47006208
Validation score: 0.787602
Iteration 116, loss = 0.46995810
Validation score: 0.781883
Iteration 117, loss = 0.46940581
Validation score: 0.782111
Iteration 118, loss = 0.47011764
Validation score: 0.787258
Iteration 119, loss = 0.46989648
Validation score: 0.786229
Iteration 120, loss = 0.47079825
Validation score: 0.780281
Iteration 121, loss = 0.46837904
Validation score: 0.787030
Iteration 122, loss = 0.46823160
Validation score: 0.783827
Iteration 123, loss = 0.46950564
Validation score: 0.783713
Iteration 124, loss = 0.46872501
Validation score: 0.786458
Iteration 125, loss = 0.46785981
Validation score: 0.784056
Iteration 126, loss = 0.46744217
Validation score: 0.787487
Validation score did not improve more than tol=0.000100 for 10 consecutiv